In [1]:
import findspark
findspark.init()

import py4j
import pyspark

from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.context import SparkContext

sc = SparkContext().getOrCreate()

sqlContext = SQLContext(sc)

#file_location = 'gs://nyctaxi-bucket1'
#https://s3.amazonaws.com/nyc- tlc/trip+data/yellow_tripdata_2017-02.csv
file_location = 'sampleYellowTripData.csv'
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true').load(file_location)

# RUN BELOW FOR THE FIRST TIME
#df.write.parquet("gs://nyctaxi-bucket1/parquet")
#df.write.save("nyctaxi-bucket1.parquet", format="parquet")
parquetDF = sqlContext.read.parquet("nyctaxi-bucket1.parquet")

parquetDF.show()

parquetDF.createOrReplaceTempView("rides")
sqlDf = sqlContext.sql("SELECT * FROM rides")

df_with_id = sqlDf.withColumn("id", monotonically_increasing_id()+1)
df_with_id = df_with_id.na.drop(subset=["tpep_pickup_datetime"])
dateFormat="MM/dd/YYYY"

df_with_pdt = df_with_id.withColumn("pdt", from_unixtime(unix_timestamp(df_with_id["tpep_pickup_datetime"]),dateFormat))
df_with_ddt = df_with_pdt.withColumn("ddt", from_unixtime(unix_timestamp(df_with_id["tpep_dropoff_datetime"]),dateFormat))

joindf = sqlContext.sql("SELECT * FROM rides t1 join rides t2 on ABS(UNIX_TIMESTAMP(t1.tpep_pickup_datetime)-UNIX_TIMESTAMP(t2.tpep_pickup_datetime)) <=300")
joindf.show()



+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1|      01/01/19 00:46|       01/01/19 00:53|              1|          1.5|         1|                 N|         151|         239|           1|          7|  0.5|    0.5|      1.65|           0|                  0.3